In [1]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import string
from IPython.display import clear_output

def list_spliter_by_num(target_list, number):
    """
    넣은 리스트를 number 단위로 쪼개는 함수
    """
    for i in range(0, len(target_list), number):
        yield target_list[i:i+number]

async def get_text_from_url(url):
    async with aiohttp.ClientSession() as session: # 다 끝나면 세션을 닫을것이다.
        try :
            async with session.get(url, headers={'user-agent': 'Mozilla/5.0'}) as req: # 다 끝나면 get 요청도 닫을것이다.
                html = await req.text() # 여기서 시간이 오래 걸리므로 기다린다.
        except Exception as e :
            print('sessiong get Error : ', e)
            main_txt = np.nan
            return url, main_txt
    
    soup = BeautifulSoup(html, 'html.parser')
    
    try :
        main_txt = soup.select_one('div#textBody').get_text()
    except Exception as e :
        main_txt = np.nan
    
    return url, main_txt
    
async def main():
    # 저장할 파일 헤더 리스트
    header_list = ['해당 기사 URL', '기사 내용']
    # 저장할 경로
    filepath = './NewsData/mt2/'
    
    try :
        news_df = pd.read_csv(filepath + 'mt_news_url.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
#         news_df = pd.read_csv(filepath + 'mt_news_url_part1.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
        try :
            main_df = pd.read_csv(filepath + 'mt_news_main.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
        except Exception as e :
            main_df = pd.DataFrame(columns= header_list)
            main_df.to_csv(filepath + 'mt_news_main.csv', encoding = 'utf-8', index = False)
        
        # 기사 url 중복 제거
        copy_df = news_df.drop_duplicates(['해당 기사 URL'])
        copy_df.set_index('해당 기사 URL', inplace = True)
        search_df = copy_df.loc[set(news_df['해당 기사 URL']) - set(main_df['해당 기사 URL'])]
        search_df.reset_index(inplace = True)

        url_list = search_df['해당 기사 URL'].dropna().tolist()
        split_list = list_spliter_by_num(url_list, 10)
        
        print('남은 기사수 : ', len(url_list))
        
        # 진행상황 체크
        progress = 0
        
        while True :
#             if progress > 200 : break
            if progress % 200 == 0 : 
                clear_output(wait = True)
                
            mt_list = [] # 크롤링된 데이터 리스트
            try :
                try :
                    tasks = []
                    try : 
                        urls = next(split_list)
                        for url in urls :
                            tasks.append(get_text_from_url(url))
                        
                        result = await asyncio.gather(*tasks)
#                         print('크롤링 결과 : ', result)
                        # 리스트에 담아주기
                        for idx in range(0, 10) :
#                             print([result[idx][0], result[idx][1]])
                            mt_list.append([result[idx][0], result[idx][1]]) # 0 : url, 1 : 본문 내용
                    except StopIteration as s :
                        return 1
                    except Exception as e :
                        print('whlie문 에러 : ', e)
                        print('=' * 60)
                        # 파일 저장
                    mt_df = pd.DataFrame(mt_list, columns = header_list)
                    mt_df.to_csv(filepath + 'mt_news_main.csv', mode = 'a', index = False, encoding = 'utf-8', header = False)
                    progress += 10
                    print('{} 기사 수집완료'.format(progress))
                except StopIteration :
                    print('모든 URL에 대한 수집이 완료되었습니다.')
                    return 1
            except Exception as e :
                print('에러 : ', e)
                return 0
    except Exception as e : 
        print('파일읽기 실패 : ', e)

In [2]:
#####################################################
############### 기사 내용 수집부분 ###################
#####################################################
while True :
    clear_output(wait = True)
    start = time.time()
    result = await main()
    end = time.time()
    
    print(f'걸린 시간 : {end-start}')
    if result == 1 :
        break
    else :
        continue

sessiong get Error :  'euc_kr' codec can't decode byte 0xc1 in position 80586: illegal multibyte sequence
19210 기사 수집완료
19220 기사 수집완료
19230 기사 수집완료
19240 기사 수집완료
19250 기사 수집완료
19260 기사 수집완료
sessiong get Error :  'euc_kr' codec can't decode byte 0x89 in position 79781: illegal multibyte sequence
19270 기사 수집완료
sessiong get Error :  'euc_kr' codec can't decode byte 0x83 in position 79235: illegal multibyte sequence
sessiong get Error :  'euc_kr' codec can't decode byte 0xa4 in position 79210: illegal multibyte sequence
sessiong get Error :  'euc_kr' codec can't decode byte 0xa6 in position 82355: illegal multibyte sequence
19280 기사 수집완료
sessiong get Error :  'euc_kr' codec can't decode byte 0xab in position 99: illegal multibyte sequence
19290 기사 수집완료
sessiong get Error :  'euc_kr' codec can't decode byte 0x89 in position 80113: illegal multibyte sequence
19300 기사 수집완료
19310 기사 수집완료
19320 기사 수집완료
sessiong get Error :  'euc_kr' codec can't decode byte 0x8a in position 80551: illegal multib

### Test

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

filepath = './NewsData/mt2/'

df = pd.read_csv(filepath + 'mt_news_url.csv', encoding = 'utf-8')

In [3]:
df.head()

,언론사명,기업명,종목코드,기사 업로드 날짜,기사 제목,해당 기사 URL,기사 연도
0,머니투데이,동화약품,20,2019-08-20,"한미약품, 상반기 R&D 투자액 사상 첫 1000억 넘어",https://news.mt.co.kr/mtview.php?no=2019081914...,2019
1,머니투데이,동화약품,20,2019-04-10,이들이 세운건 기업이 아닌 나라였다,https://news.mt.co.kr/mtview.php?no=2019040912...,2019
2,머니투데이,동화약품,20,2019-04-10,"""나라 없이 기업 없다""…100년 전 민족기업의 투혼",https://news.mt.co.kr/mtview.php?no=2019040910...,2019
3,머니투데이,동화약품,20,2019-04-17,"""그들이 이익에 원금까지 나라를 위해 낼 수 있었던 건…""",https://news.mt.co.kr/mtview.php?no=2019041516...,2019
4,머니투데이,동화약품,20,2019-05-16,"122년 제약 외길 '동화약품', 신약으로 100년 대계",https://news.mt.co.kr/mtview.php?no=2019051414...,2019


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354277 entries, 0 to 1354276
Data columns (total 7 columns):
언론사명         1354277 non-null object
기업명          1354277 non-null object
종목코드         1354277 non-null int64
기사 업로드 날짜    1353799 non-null object
기사 제목        1353799 non-null object
해당 기사 URL    1353799 non-null object
기사 연도        1354277 non-null int64
dtypes: int64(2), object(5)
memory usage: 72.3+ MB


In [5]:
len(df['해당 기사 URL'].unique())

593437

In [6]:
df.loc[1, '해당 기사 URL']

'https://news.mt.co.kr/mtview.php?no=2019040912027621738&type=1'

In [15]:
# url = 'https://news.mt.co.kr/mtview.php?no=2019081914364439807&type=1'
url = 'https://news.mt.co.kr/mtview.php?no=2019040912027621738&type=1'

res = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(res.text, 'html.parser')
soup.select_one('div#textBody').get_text()

'\n지난 3일 중국 상하이 황포구 대한민국 임시정부 외관 모습/사진=이건희 기자\n\n# 지난 3일 오전 중국 상하이 황포구 마당로에 위치한 대한민국 임시정부(임정)청사. 이곳은 주변과 이질적인 공간이었다. 청사가 있는 곳은 상하이 황포구 쇼핑가 신톈디(신천지)역 근처로, 서울의 압구정과 같은 곳이다. 100년 전인 1919년에 세워진 임정 기념관은 신톈디역에서 몇 걸음 떨어지지 않은 곳에 있었다. 기념관 입구 주변엔 전기오토바이들이 주차돼있고, 빨래가 널려 있었다. 100년 전 백범 김구와 임정 요인들의 공간은 허름했다. 등받이 없는 나무의자, 가파른 내부 계단, 어른 키 수준의 작은 화장실 공간 등 나라잃은 민족의 슬픔이 서려있었다. 100년이 지나 주변 지역은 천지개벽했지만, 이곳은 시간이 그대로 멈춘 듯 했다.\n\r\n# 같은 시각 중국 충칭 위중구 연화지에 있는 대한민국 임정청사. 이봉창, 윤봉길 의사 의거 후 일제의 박해가 심해지자 임정청사는 항저우, 자싱, 전장, 난징, 창사, 광저우 등 중국 각지를 전전하다 1940년 4월 이곳에 자리를 잡았다. 1945년 8월15일 광복때까지 운영된 마지막 임정청사다. 상하이 청사의 12배 규모인 이곳은 법무부와 재무부(현 기획재정부) 등 각 부처 사무실이 있었다. 임정은 21년간 제법 체계가 갖춰졌다. 계단 양옆 건물 각 층엔 회의실 10여개와 외빈을 맞이하는 숙소 등이 있었다. 전시실엔 당시 대한민국이 정상국가로 운영됐음을 증명하는 각종 공문서들이 그대로 보존됐다. 이곳에서 대한민국의 뼈대가 형성됐다.\n\r\n1919년 임정의 혼은 100년이란 시간을 타고 지금의 대한민국을 만들었다. 세계에서 가장 가난했던 나라가 이제 세계 10위권의 경제대국이 됐다. 1919~1945년, 대한민국의 시작을 이끌었던 임정은 어떻게 운영됐을까. 또 독립운동은 어떤 돈으로 할 수 있었을까.\n\r\n당시 미국과 중국 등 해외에 나간 국민과 국내에 거주했던 국민이 십시일반 돈을 모아 임정 운영과 독립운동 자금으로 보탰고, 

### 본문 내용 크롤링 데이터 확인

In [7]:
main_check_df = pd.read_csv(filepath + 'mt_news_main.csv', encoding = 'utf-8')

In [8]:
main_check_df.head()

,해당 기사 URL,기사 내용
0,https://news.mt.co.kr/mtview.php?no=2019030215...,\n2006 독일 월드컵 한국-프랑스전 당시 지네딘 지단(가운데)과 박지성(오른쪽)...
1,https://news.mt.co.kr/mtview.php?no=2017053113...,\n\n중소기업청과 중소기업기술정보진흥원(이하 기정원)은 올해 산학연협력기술개발사업...
2,https://news.mt.co.kr/mtview.php?no=2019012810...,\n한화 이글스 안영명이 25일 대전시 중구에 위치한 아동복지시설인 늘사랑 아동센터...
3,https://news.mt.co.kr/mtview.php?no=2018021411...,\n14일 오전 경기도 의왕시 서울구치소를 찾은황각규 롯데그룹 부회장 /사진=박진영...
4,https://news.mt.co.kr/mtview.php?no=2017040415...,\n 문재인 더불어민주당 대선 후보와 추미애 대표가 4일 서울 여의도 국회에서 열린...


In [11]:
test_index = 1

In [12]:
main_check_df.loc[test_index, '해당 기사 URL']

'https://news.mt.co.kr/mtview.php?no=2017053113475373757&type=1'

In [13]:
main_check_df.loc[test_index, '기사 내용']

'\n\n중소기업청과 중소기업기술정보진흥원(이하 기정원)은 올해 산학연협력기술개발사업 중 전략협력(산연전용, 연구마을) 상반기 과제 161개를 최종 선정했다고 31일 밝혔다.\n\r\n전략협력 사업은 특정 산업·기술분야에서 전략적 산학연협력체계를 구축한 뒤 유망 중소기업을 발굴해 고성장기업으로 육성하는 사업이다. 연구기관의 인적·물적 인프라를 활용해 중소기업의 기술혁신 역량을 강화하는 산연전용은 1년에 1억5000만원씩, 중소기업의 연구기능(부설연구소)을 대학 내 공간에 집적화해 상시 기술협력체계를 구축하는 연구마을은 2년에 2억원씩 지원한다.\n\r\n기정원은 상반기 산연전용·연구마을사업에 신청한 기업 346개를 대상으로 2단계 심층평가를 거쳐 161개 과제, 247억원 지원을 결정했다.\n\r\n아울러 기정원은 상반기 산연전용·연구마을 과제에 선정된 주관기관과 공동개발기관(대학·연구기관)을 대상으로 다음달 1일 충북대학교 개신문화관에서 협약설명회를 개최한다. \n\r\n설명회는 선정기업의 대표자, 과제책임자, 실무자 및 공동개발기관인 대학·연구기관의 공동책임자 등을 대상으로 사업추진체계 및 절차, 수정사업계획서 작성, 전자협약 등 사업추진에 필요한 제반사항을 안내한다.\n\r\n최광문 기정원 협력기술평가본부장은 "참여하는 모든 중소기업이 대학·연구기관의 전략적 지원프로그램을 잘 활용하여 고성장기업으로 발전할 수 있는 기틀을 마련하기 바란다”고 밝혔다.                            '

### 데이터셋 두개로 나누기

In [4]:
import pandas as pd
import numpy as np

filepath = './NewsData/mt2/'

news_df = pd.read_csv(filepath + 'mt_news_url.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
try :
    main_df = pd.read_csv(filepath + 'mt_news_main.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
except Exception as e :
    main_df = pd.DataFrame(columns= header_list)
    main_df.to_csv(filepath + 'mt_news_main.csv', encoding = 'utf-8', index = False)

# 기사 url 중복 제거
copy_df = news_df.drop_duplicates(['해당 기사 URL'])
copy_df.set_index('해당 기사 URL', inplace = True)
search_df = copy_df.loc[set(news_df['해당 기사 URL']) - set(main_df['해당 기사 URL'])]
search_df.reset_index(inplace = True)

In [5]:
len(news_df['해당 기사 URL'].unique())

593437

In [6]:
len(main_df['해당 기사 URL'].unique())

155770

In [7]:
len(search_df['해당 기사 URL'])

437667

In [8]:
len(news_df['해당 기사 URL'].unique()) - len(main_df['해당 기사 URL'].unique())

437667

In [19]:
len_num = int(len(search_df['해당 기사 URL']) / 2) + 1

df1 = search_df[ : len_num ]
df2 = search_df[len_num : ]

In [20]:
df1.tail()

,해당 기사 URL,언론사명,기업명,종목코드,기사 업로드 날짜,기사 제목,기사 연도
218829,https://news.mt.co.kr/mtview.php?no=2017011815...,머니투데이,대우조선해양,042660,2017.01.18 16:14,"'5조원대 분식회계' 고재호 前 대우조선 사장, 1심서 징역 10년",2017
218830,https://news.mt.co.kr/mtview.php?no=2017042611...,머니투데이,대상,001680,2017.04.26 11:38,"편의점에 원터치 신고시스템 구축…BGF, 경찰청과 협약",2017
218831,https://news.mt.co.kr/mtview.php?no=2019020715...,머니투데이,SBS,034120,2019-02-07,"SBS미디어홀딩스, 작년 영업익 71억… '흑자전환'",2019
218832,https://news.mt.co.kr/mtview.php?no=2017052311...,머니투데이,SK하이닉스,000660,2017-05-23,"[오늘의포인트]코스피 2300, 시총 변화는",2017
218833,https://news.mt.co.kr/mtview.php?no=2017101716...,머니투데이,신흥,004080,2017.10.18 17:36,"세계로TV 김원기 대표, “아시아 신흥국 시장 주목해야”",2017


In [21]:
df2.head()

,해당 기사 URL,언론사명,기업명,종목코드,기사 업로드 날짜,기사 제목,기사 연도
218834,https://news.mt.co.kr/mtview.php?no=2017090416...,머니투데이,부산산업,011390,2017.09.04 16:11,"기보, 경기도서 6번째 현장간담회 개최",2017
218835,https://news.mt.co.kr/mtview.php?no=2018030212...,머니투데이,대상,001680,2018.03.02 16:26,"리맥스(RE/MAX), 국내 최초 '영문 물건 정보 서비스' 론칭",2018
218836,https://news.mt.co.kr/mtview.php?no=2013111516...,머니투데이,현대증권,003450,2013.11.15 16:31,[수익률게임] 1위 노리는 포트폴리오 신규 편입 종목은?,2013
218837,https://news.mt.co.kr/mtview.php?no=2018042315...,머니투데이,LG,003550,2018.04.23 16:17,"골프존, 베트남 진출…아시아 시장 교두보",2018
218838,https://news.mt.co.kr/mtview.php?no=2018080715...,머니투데이,SK하이닉스,000660,2018-08-07,투심 약해지는 IT… 그래도 오를 종목은?,2018


In [22]:
df1.to_csv(filepath + 'mt_news_url_part1.csv', encoding = 'utf-8', index = False)
df2.to_csv(filepath + 'mt_news_url_part2.csv', encoding = 'utf-8', index = False)

In [24]:
len(df1), len(df2)

(218834, 218833)